In [ ]:
import torch
#from segment_anything import SamPredictor, sam_model_registry
from PIL import Image
import numpy as np
import kagglehub


In [ ]:


# Download latest version
path = kagglehub.model_download("metaresearch/segment-anything/pyTorch/vit-b")

print("Path to model files:", path)

100%|██████████| 358M/358M [00:12<00:00, 29.7MB/s]


Path to model files: C:\Users\User\.cache\kagglehub\models\metaresearch\segment-anything\pyTorch\vit-b\1


In [ ]:
import os
import numpy as np
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry

# Caminho para o modelo baixado
model_path = "C:/Users/User/.cache/kagglehub/models/metaresearch/segment-anything/pyTorch/vit-b/1/model.pth"

# Carregar o modelo
sam = sam_model_registry["vit_b"](checkpoint=model_path)

# Configurar o preditor
predictor = SamPredictor(sam)

# Caminhos para os diretórios
base_dir = "D:\\T.I\\BootcampAvanti_ML\\DeteccaoMelanoma_ML"
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

masks_train_dir = os.path.join(base_dir, 'masks_train')
masks_test_dir = os.path.join(base_dir, 'masks_test')

classes = ['Benign', 'Malignant']

# Função para carregar e preparar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

# Função para gerar máscaras
def generate_masks(image):
    predictor.set_image(image)
    masks, scores, logits = predictor.predict()
    return masks

# Função para salvar as máscaras
def save_masks(masks, output_dir, img_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, mask in enumerate(masks):
        mask_image = Image.fromarray((mask * 255).astype(np.uint8))
        mask_image.save(os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}_mask_{i}.png"))

# Função para processar imagens de um diretório e salvar máscaras
def process_images(input_dir, output_dir, sample_size=5):
    for class_name in classes:
        class_dir = os.path.join(input_dir, class_name)
        dest_class_dir = os.path.join(output_dir, class_name)

        if not os.path.exists(dest_class_dir):
            os.makedirs(dest_class_dir)

        # Pegue uma amostra de imagens
        img_names = os.listdir(class_dir)
        sample_img_names = img_names[:sample_size]

        for img_name in sample_img_names:
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path):
                try:
                    # Carregar e preparar a imagem
                    img = load_image(img_path)

                    # Gerar máscaras
                    masks = generate_masks(img)

                    # Salvar as máscaras
                    save_masks(masks, dest_class_dir, img_name)

                    print(f"Máscaras salvas para {img_name} em {dest_class_dir}")
                except Exception as e:
                    print(f"Erro ao processar a imagem {img_name}: {e}")

# Processar imagens de treino e teste
process_images(train_dir, masks_train_dir, sample_size=5)
process_images(test_dir, masks_test_dir, sample_size=5)


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Máscaras salvas para 1.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Benign
Máscaras salvas para 10.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Benign
Máscaras salvas para 100.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Benign
Máscaras salvas para 1000.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Benign
Máscaras salvas para 1001.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Benign
Máscaras salvas para 1.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Malignant
Máscaras salvas para 10.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Malignant
Máscaras salvas para 100.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Malignant
Máscaras salvas para 1000.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Malignant
Máscaras salvas para 1001.jpg em D:\T.I\BootcampAvanti_ML\DeteccaoMelanoma_ML\masks_train\Malignant
Máscaras salvas para 6299.j

In [ ]:
def predict_and_save_masks(model, input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        if os.path.isfile(img_path):
            try:
                # Carregar e pré-processar a imagem
                img = cv2.imread(img_path)
                preprocessed = preprocess_image(img)

                # Fazer previsão
                mask = model.predict(preprocessed)[0]

                # Aplicar o threshold
                mask = (mask > 0.5).astype(np.uint8) * 255

                # Salvar a máscara
                dest_path = os.path.join(output_dir, img_name)
                cv2.imwrite(dest_path, mask)
                print(f"Máscara salva em: {dest_path}")
            except Exception as e:
                print(f"Erro ao processar a imagem {img_name}: {e}")
